In [1]:
import numpy as np
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import PandasTools
import torch
import pandas as pd
import os
import importlib
import nglview

_ColormakerRegistry()

In [2]:
%load_ext autoreload
%autoreload 2

###  Conformation Generation

In [2]:
import DataGen
from DataGen import genconfs
from DataGen import util
from DataGen.genconfs import runGenerator
from DataGen.util import get_RMSD

The main function here is runGenerator

In [7]:
help(genconfs.runGenerator)

Help on function runGenerator in module DataGen.genconfs:

runGenerator(index_list, smiles_list, source_data_name, datadir, structure_dir=None, numConfs=300, clusterMethod='RMSD', clusterThreshold=0.2)
    Generate conformation as sdf for all smiles in input
    index_list: the list of initial indexes from source data 
    smiles_list: the rdkit smiles generated using initial source data SMILES
    source_data_name: the source data name, such as ccdc, zinc, and pubchem, which is used to record the initial index in that source data
    datadir: directory for outputfile
    structure_dir: the directory used to find the 3D structure file, if it is None, the numConfs should not be None
    numConfs: the number of conformations generated before clustering, defaults to 300; if numConfs=None, we conducted local MMFF minimization for structure
    clusterMethod: the distance calculation method used in clustering, defaults to RMSD
    clusterThreshold: the clustering threshold, defaults to 0.2


For variables such as clusterMethod and clusterThreshold,, we can always use default values

#### Conformation generation for smiles with no structure reference

In [8]:
### I use the first molecule in Index_ccdc_20_overlapwithmol20_removeempty.csv ###
index_list = ["33"]
smiles_list = ["COc1ccc(C2OC(=O)C(C)C2(C)CC(C)C)cc1"]
source_data_name = "ccdc"
structure_dir = None
numConfs = 300
%mkdir "../test/confs_smiles"
datadir = "../test/confs_smiles"
Failed_list = genconfs.runGenerator(index_list, smiles_list, source_data_name, datadir, structure_dir, numConfs)

mkdir: ../test/confs_smiles: File exists
0 COc1ccc(C2OC(=O)C(C)C2(C)CC(C)C)cc1
The total number of conformers after clustring: 254


In [9]:
Failed_list

[]

After conformation genertaion, the generated conformation(254 conformations in this case) has been saved in a 33_confors.sdf in ../test/confs_smiles. <br>The return of this function is the list for all failed cases (idx of index_list), there it's empty since no failed case.

#### Conformation generation for smiles with structure reference

This time we use reference structure of this SMILES from ccdc dataset.<br>
The reason to do this is that we want to make sure all newly generated conformations have right protonation state.<br>
When we have crystal structure, we should use crystal structure as reference to generate conformations

In [8]:
index_list = ["33"]
smiles_list = ["COc1ccc(C2OC(=O)C(C)C2(C)CC(C)C)cc1"]
source_data_name = "ccdc"
structure_dir = "../test/reference_structure"
numConfs = 300
%mkdir "../test/confs_smiles_withreference"
datadir = "../test/confs_smiles_withreference"
Failed_list = genconfs.runGenerator(index_list, smiles_list, source_data_name, datadir, structure_dir, numConfs)

0 COc1ccc(C2OC(=O)C(C)C2(C)CC(C)C)cc1
Use 3D Structure as Reference: 33.sdf
The total number of conformers after clustring: 248


#### Local optimization for reference structure

To conduct the local minimization, we don't need conformation generation, and hence the numConfs should be 0 

In [32]:
index_list = ["33"]
smiles_list = ["COc1ccc(C2OC(=O)C(C)C2(C)CC(C)C)cc1"]
source_data_name = "ccdc"
structure_dir = "../test/reference_structure"
numConfs = 0
%mkdir "../test/confs_local_minimization"
datadir = "../test/confs_local_minimization"
Failed_list = genconfs.runGenerator(index_list, smiles_list, source_data_name, datadir, structure_dir, numConfs)

0 COc1ccc(C2OC(=O)C(C)C2(C)CC(C)C)cc1
Use 3D Structure as Reference: 33.sdf
Finish Local Minimization with MMFF


#### Check generated conformations

In [3]:
### Check Attribute
conf_1 = PandasTools.LoadSDF("../test/confs_smiles/33_confors.sdf")
conf_2 = PandasTools.LoadSDF("../test/confs_smiles_withreference/33_confors.sdf")
conf_1.head()
conf_2.head()

,energy_abs,SMILES,cluster_no,ccdc_id,initial_conformation_id,minimize_method,ID,ROMol
0,44.5737028374007,COc1ccc(C2OC(=O)C(C)C2(C)CC(C)C)cc1,1,33,283,MMFF,,"<img src=""data:image/png;base64,iVBORw0KGgoAAA..."
1,44.82607464748033,COc1ccc(C2OC(=O)C(C)C2(C)CC(C)C)cc1,2,33,258,MMFF,,"<img src=""data:image/png;base64,iVBORw0KGgoAAA..."
2,46.0972404834002,COc1ccc(C2OC(=O)C(C)C2(C)CC(C)C)cc1,3,33,235,MMFF,,"<img src=""data:image/png;base64,iVBORw0KGgoAAA..."
3,44.54444482507678,COc1ccc(C2OC(=O)C(C)C2(C)CC(C)C)cc1,4,33,214,MMFF,,"<img src=""data:image/png;base64,iVBORw0KGgoAAA..."
4,43.55545027087747,COc1ccc(C2OC(=O)C(C)C2(C)CC(C)C)cc1,5,33,154,MMFF,,"<img src=""data:image/png;base64,iVBORw0KGgoAAA..."


,energy_abs,SMILES,cluster_no,ccdc_id,initial_conformation_id,minimize_method,ID,ROMol
0,44.925354442201694,COc1ccc(C2OC(=O)C(C)C2(C)CC(C)C)cc1,1,33,260,MMFF,ABABAH,"<img src=""data:image/png;base64,iVBORw0KGgoAAA..."
1,43.905396003013806,COc1ccc(C2OC(=O)C(C)C2(C)CC(C)C)cc1,2,33,254,MMFF,ABABAH,"<img src=""data:image/png;base64,iVBORw0KGgoAAA..."
2,44.835954486888966,COc1ccc(C2OC(=O)C(C)C2(C)CC(C)C)cc1,3,33,191,MMFF,ABABAH,"<img src=""data:image/png;base64,iVBORw0KGgoAAA..."
3,44.16119342798035,COc1ccc(C2OC(=O)C(C)C2(C)CC(C)C)cc1,4,33,278,MMFF,ABABAH,"<img src=""data:image/png;base64,iVBORw0KGgoAAA..."
4,50.24077902861522,COc1ccc(C2OC(=O)C(C)C2(C)CC(C)C)cc1,5,33,274,MMFF,ABABAH,"<img src=""data:image/png;base64,iVBORw0KGgoAAA..."


In [4]:
### Check RMSD
reference_1 = Chem.SDMolSupplier("../test/reference_structure/33.sdf")
reference_2 = Chem.SDMolSupplier("../test/confs_local_minimization/33_min.sdf")
prob_1 = Chem.SDMolSupplier("../test/confs_smiles/33_confors.sdf")
prob_2 = Chem.SDMolSupplier("../test/confs_smiles_withreference/33_confors.sdf")
get_RMSD(reference_1[0], reference_2[0])
get_RMSD(prob_2[0], reference_2[0])
get_RMSD(prob_1[0], reference_2[0])

0.1510568782980715

1.1748416601353016

1.323696229379208

In [5]:
### View Structure: can only show the first one

nglview.show_file("../test/confs_smiles/33_confors.sdf")
nglview.show_file("../test/confs_smiles_withreference/33_confors.sdf")

NGLWidget()

NGLWidget()

### Generate Gaussian Input

In [4]:
import DataGen
from DataGen import genGaus
from DataGen.genGaus import file_seperate
from DataGen.genGaus import file_transfer
from DataGen.genGaus import gaussian_gen

#### Generate gaussian input for local minimized structure ( sdf file with only one conformation)

In [5]:
index_list = ["33"]
### if ftype is cry --> *_min.sdf; else: *.sdf 
ftype = "cry"
header_file = "../test/confs_local_minimization/header.txt"
datadir = "../test/confs_local_minimization/"
file_transfer(index_list, ftype, datadir)
gaussian_gen(index_list, header_file, datadir)

In [7]:
### Check the output
ginput = open("../test/confs_local_minimization/33.opt.com").readlines()
ginput

['%NProcShared=1\n',
 '%Mem=10GB\n',
 '\n',
 '#p B3LYP/6-31G* opt freq\n',
 '\n',
 'test\n',
 '\n',
 '0  1\n',
 'O           2.98360         2.50210        13.57860\n',
 'O           3.70340         2.09020        15.70480\n',
 'O           1.49550         1.85930         7.39580\n',
 'C           2.80590         2.35140        14.92170\n',
 'C           1.35720         2.58870        15.26940\n',
 'H           1.27470         3.65490        15.52350\n',
 'C           0.64780         2.31320        13.91390\n',
 'C           1.73970         2.91640        12.99030\n',
 'H           1.73780         4.01420        13.07000\n',
 'C           0.91260         1.77830        16.47680\n',
 'H          -0.17200         1.82760        16.61180\n',
 'H           1.37880         2.17440        17.38630\n',
 'H           1.21720         0.72890        16.40730\n',
 'C          -0.65520         3.11850        13.84360\n',
 'H          -1.37150         2.77230        14.59580\n',
 'H          -1.127

#### Generate gaussian input for conformations

In [9]:
infile_list = ["33_confors.sdf"]
datadir = "../test/confs_smiles/"
%mkdir "../test/confs_smiles/ginput"
outdir = "../test/confs_smiles/ginput/"
### use this initial index, we can reorder all conformations just based on conformation orders
initial_index = 0
### if initial_index = 0 , the conformations will start from 1 
final_index = file_seperate(infile_list, datadir, outdir, initial_index)

In [10]:
final_index
### this will be next initail_index for next molecule

254

In [11]:
index_list = [i for i in range(1, 255)]
### if ftype is cry --> *_min.sdf; else: *.sdf 
ftype = "conformations"
header_file = "../test/confs_local_minimization/header.txt"
datadir = "../test/confs_smiles/ginput/"
file_transfer(index_list, ftype, datadir)
gaussian_gen(index_list, header_file, datadir)

**Warning** The file transfer is based on obabel, and it takes some time to finish, and don't do this transfermation for so many conformations at the same time on your own computer, it can cause problem 

In [12]:
### Check the output
ginput = open("../test/confs_smiles/ginput/1.opt.com").readlines()
ginput

['%NProcShared=1\n',
 '%Mem=10GB\n',
 '\n',
 '#p B3LYP/6-31G* opt freq\n',
 '\n',
 'test\n',
 '\n',
 '0  1\n',
 'C           4.98580         2.46560         3.31510\n',
 'O           4.14670         1.34220         3.55200\n',
 'C           3.04200         1.21860         2.75580\n',
 'C           2.23710         0.11500         3.04030\n',
 'C           1.07590        -0.13240         2.30290\n',
 'C           0.69750         0.72070         1.25690\n',
 'C          -0.55890         0.43040         0.45960\n',
 'O          -1.08480         1.62800        -0.12300\n',
 'C          -1.84580         1.29440        -1.20000\n',
 'O          -2.53450         2.09250        -1.81480\n',
 'C          -1.72100        -0.18350        -1.49790\n',
 'C          -1.72730        -0.40950        -3.00500\n',
 'C          -0.44480        -0.57960        -0.72150\n',
 'C           0.84100        -0.32340        -1.53630\n',
 'C          -0.48370        -2.02980        -0.15170\n',
 'C          -0.822

### Generate Jobs for HPC

In [5]:
import DataGen
from DataGen import genjob_hpc
from DataGen.genjob_hpc import gen_job
from DataGen.genjob_hpc import sub_job
from DataGen.genjob_hpc import check_gaussian

#### Generate jobs

In [4]:
index_list = [i for i in range(1,255)]
### the number of jobs that have been generated 
current_jobs = 20
index_list
%mkdir "../test/confs_smiles/gjobs"
jobdir = "../test/confs_smiles/gjobs"
datadir = "/beegfs/jl7003/1Dto3D_DataGen/test/confs_smiles/ginput"
### the number of calculations in each job
num_jobs=10
### time requried for each job
time=5
### memory required for each job
mem=10
gen_job(index_list, current_jobs, jobdir, datadir,  num_jobs=10, time=5, mem=10)

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

mkdir: ../test/confs_smiles/gjobs: File exists


Try different time, memory combination.<br>
Notice: in HPC, job has small time and memory requirement don't need to wait for lone time. For large molecule, such as molecule with 20 heavy atoms, you need only contain one molecule in the job. For small molecule, such as molecule with 10 heavy atoms, you can make more than one molecule in the job.

#### Submit jobs

In [6]:
### Submit jobs
job_list = [i for i in range(1,24)]
jobdir = "../test/confs_smiles/gjobs"
### job range is determined by id1 and id2 
id1 = 0
id2 = 2
sub_job(job_list, id1, id2)

In HPC, we can at most submit 1000 jobs, but only 200 jobs can run at the same time.

#### Check output: error or normal

In [6]:
help(check_gaussian)

Help on function check_gaussian in module DataGen.genjob_hpc:

check_gaussian(index_list, datadir)
    Check whether the gaussian calculation finishes normally



In [9]:
### Here, we used the log file from 33_min.sdf as an example
### The file is 33.opt.log in ../test/confs_local_minimization
### Gaussian_error.csv will be saved in current dir
index_list = ["33"]
datadir = "../test/confs_local_minimization/"
check_gaussian(index_list, datadir)

In [11]:
error = open("Gaussian_error.csv").readline()

In [12]:
error

''

Since our calculation finishes normally, there is no record in error file 

### Analysis

In [3]:
import DataGen
from DataGen import analysis
from DataGen.analysis import convert_tosdf
from DataGen.analysis import getInfor

Here, we conducted consistency check, to see if our initial SMILES, MMFF optimized structure, and QM optimized structure are consistent

#### Consistency Check 

In [16]:
### first, we need to convert Gaussian output into sdf 
datadir = "../test/confs_local_minimization/"
index = "33"
### if a output file can't be converted back into sdf file, we should remove it 
convert_tosdf(datadir, index)

In [26]:
infile1 = "COc1ccc(C2OC(=O)C(C)C2(C)CC(C)C)cc1"
infile2 = "../test/confs_local_minimization/33_min.sdf"
infile3 = "../test/confs_local_minimization/33.opt.sdf"
### here, we consider SMILES with no stereochemistry
isomericSmiles = False
infile = getInfor(infile1, infile2, infile3, isomericSmiles)


In [28]:
infor_df = pd.DataFrame(columns=infile.inforname, data=[infile.infor])
infor_df
infor_df.to_csv("Infor_noisomeric.csv", index=None)

,SMILES,initial_InChI,initial_SMILES,MMFF_InChI,MMFF_SMILES,QM_InChI,QM_SMILES
0,COc1ccc(C2OC(=O)C(C)C2(C)CC(C)C)cc1,InChI=1S/C17H24O3/c1-11(2)10-17(4)12(3)16(18)2...,COc1ccc(C2OC(=O)C(C)C2(C)CC(C)C)cc1,InChI=1S/C17H24O3/c1-11(2)10-17(4)12(3)16(18)2...,COc1ccc(C2OC(=O)C(C)C2(C)CC(C)C)cc1,InChI=1S/C17H24O3/c1-11(2)10-17(4)12(3)16(18)2...,COc1ccc(C2OC(=O)C(C)C2(C)CC(C)C)cc1


In [29]:
### Check MMFF and QM 
infor_df["MMFF_InChI"].values == infor_df["QM_InChI"].values
infor_df["MMFF_SMILES"].values == infor_df["QM_SMILES"].values

array([ True])

array([ True])

This conformation passed the consistency check 

After QM calculation, detailed check can be condected including redudency check and consistency check, which will be futher organized into script

### Prepare dataset

In [2]:
import DataGen
from DataGen import prepare_data
from DataGen.prepare_data import prepare_data
from DataGen.prepare_data import prepre_PhysNet_input
from DataGen.prepare_data import prepare_torch
from DataGen.prepare_data import prepare_target_csv

In [3]:
datadir = "../test/confs_local_minimization/"
reference = "../test/reference_method/atomref.B3LYP_631Gd.10As.npz"
index_list = ["33"]
ftype = "cry"
exp = prepare_data(index_list[0], datadir, reference, ftype)

prepare_data returns a object with all informations you need to prepare a dataset

In [4]:
exp.elementdict
exp.elementdict_periodic
exp.conversions
exp.reference[1]

{'H': 1,
 'B': 5,
 'C': 6,
 'N': 7,
 'O': 8,
 'F': 9,
 'P': 15,
 'S': 16,
 'Cl': 17,
 'Br': 35}

{'H': [1, 1],
 'B': [2, 3],
 'C': [2, 4],
 'N': [2, 5],
 'O': [2, 6],
 'F': [2, 7],
 'P': [3, 5],
 'S': [3, 6],
 'Cl': [3, 7],
 'Br': [4, 7]}

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 27.211386024367243,
 27.211386024367243,
 27.211386024367243,
 1.0,
 27.211386024367243,
 27.211386024367243,
 27.211386024367243,
 27.211386024367243,
 27.211386024367243,
 1.0,
 27.211386024367243]

array([  0.        , -13.61312172, -13.5745904 , -13.54887564,
       -13.90303183,   2.981     ])

In [5]:
exp.QMmol
exp.MMFFmol

In [6]:
exp.natoms

44

In [7]:
exp.charges_mulliken

[-0.481708,
 -0.480974,
 -0.507588,
 0.639258,
 -0.202483,
 0.1642,
 0.065918,
 0.065322,
 0.140988,
 -0.453908,
 0.146038,
 0.166539,
 0.163978,
 -0.480721,
 0.155661,
 0.160389,
 0.138807,
 -0.296947,
 0.145274,
 0.144663,
 -0.071655,
 0.14808,
 -0.450827,
 0.140908,
 0.142458,
 0.137824,
 -0.449385,
 0.138499,
 0.137223,
 0.143784,
 0.123667,
 -0.186025,
 0.131086,
 -0.180072,
 0.140842,
 0.384828,
 -0.199035,
 0.137132,
 -0.177787,
 0.154742,
 -0.219045,
 0.170052,
 0.154147,
 0.155852]

In [7]:
exp.QMcoords
exp.MMFFcoords

[[-1.2987, -1.9824, -0.2864],
 [-3.2344, -2.4613, -1.332],
 [4.765, -0.1745, 0.3102],
 [-2.6252, -1.7905, -0.5386],
 [-3.1304, -0.6465, 0.3293],
 [-3.4196, -1.1162, 1.2844],
 [-1.8311, 0.1718, 0.6188],
 [-0.8411, -1.0468, 0.7231],
 [-1.017, -1.5183, 1.7018],
 [-4.3652, 0.0414, -0.2493],
 [-4.6902, 0.8716, 0.3873],
 [-5.1845, -0.6802, -0.3184],
 [-4.1879, 0.4254, -1.2576],
 [-1.9308, 0.9141, 1.9558],
 [-2.7123, 1.681, 1.921],
 [-0.9912, 1.4091, 2.2192],
 [-2.1837, 0.2214, 2.7682],
 [-1.5083, 1.1077, -0.5798],
 [-2.4298, 1.6347, -0.8555],
 [-1.2713, 0.4679, -1.4399],
 [-0.393, 2.1809, -0.4363],
 [0.3808, 1.8132, 0.2467],
 [-0.9272, 3.5171, 0.11],
 [-0.1163, 4.2499, 0.1989],
 [-1.3916, 3.4183, 1.0954],
 [-1.6791, 3.9386, -0.5705],
 [0.2731, 2.4217, -1.8016],
 [1.0415, 3.201, -1.7338],
 [-0.4656, 2.7483, -2.5458],
 [0.7504, 1.5104, -2.1776],
 [0.6442, -0.8326, 0.5644],
 [1.4164, -0.4506, 1.6737],
 [0.9433, -0.3306, 2.6458],
 [2.7821, -0.2294, 1.5582],
 [3.3801, 0.0667, 2.4143],
 [3.4211, -

[[2.9836, 2.5021, 13.5786],
 [3.7034, 2.0902, 15.7048],
 [1.4955, 1.8593, 7.3958],
 [2.8059, 2.3514, 14.9217],
 [1.3572, 2.5887, 15.2694],
 [1.2747, 3.6549, 15.5235],
 [0.6478, 2.3132, 13.9139],
 [1.7397, 2.9164, 12.9903],
 [1.7378, 4.0142, 13.07],
 [0.9126, 1.7783, 16.4768],
 [-0.172, 1.8276, 16.6118],
 [1.3788, 2.1744, 17.3863],
 [1.2172, 0.7289, 16.4073],
 [-0.6552, 3.1185, 13.8436],
 [-1.3715, 2.7723, 14.5958],
 [-1.1278, 3.0501, 12.8599],
 [-0.4772, 4.1831, 14.0352],
 [0.4101, 0.7768, 13.7024],
 [-0.0268, 0.3493, 14.612],
 [1.3914, 0.2944, 13.6022],
 [-0.5006, 0.3253, 12.5322],
 [-0.4651, 1.0439, 11.7111],
 [-1.9647, 0.1732, 12.9635],
 [-2.5803, -0.1613, 12.1211],
 [-2.389, 1.1141, 13.319],
 [-2.0685, -0.5662, 13.7653],
 [-0.0198, -1.0262, 11.9879],
 [-0.6447, -1.3529, 11.1498],
 [-0.0571, -1.8014, 12.7612],
 [1.0106, -0.9616, 11.6251],
 [1.713, 2.5964, 11.5136],
 [0.8952, 3.342, 10.6521],
 [0.2801, 4.1533, 11.0346],
 [0.8547, 3.0578, 9.2844],
 [0.212, 3.6436, 8.6314],
 [1.6323, 2

In [8]:
exp.elements

[8,
 8,
 8,
 6,
 6,
 1,
 6,
 6,
 1,
 6,
 1,
 1,
 1,
 6,
 1,
 1,
 1,
 6,
 1,
 1,
 6,
 1,
 6,
 1,
 1,
 1,
 6,
 1,
 1,
 1,
 6,
 6,
 1,
 6,
 1,
 6,
 6,
 1,
 6,
 1,
 6,
 1,
 1,
 1]

In [9]:
exp.elements_p

[[2, 6],
 [2, 6],
 [2, 6],
 [2, 4],
 [2, 4],
 [1, 1],
 [2, 4],
 [2, 4],
 [1, 1],
 [2, 4],
 [1, 1],
 [1, 1],
 [1, 1],
 [2, 4],
 [1, 1],
 [1, 1],
 [1, 1],
 [2, 4],
 [1, 1],
 [1, 1],
 [2, 4],
 [1, 1],
 [2, 4],
 [1, 1],
 [1, 1],
 [1, 1],
 [2, 4],
 [1, 1],
 [1, 1],
 [1, 1],
 [2, 4],
 [2, 4],
 [1, 1],
 [2, 4],
 [1, 1],
 [2, 4],
 [2, 4],
 [1, 1],
 [2, 4],
 [1, 1],
 [2, 4],
 [1, 1],
 [1, 1],
 [1, 1]]

In [10]:
exp.init_target

[0.54211,
 0.21322,
 0.18612,
 3.7715,
 181.74,
 -0.00816,
 -0.21982,
 0.21166,
 6314.4973,
 0.383215,
 -887.559757,
 -887.539494,
 -887.538549,
 -887.607685,
 78.479,
 -887.942971468]

In [11]:
exp.target

[0.54211,
 0.21322,
 0.18612,
 3.7715,
 181.74,
 -0.22204490995883672,
 -5.981606875876407,
 5.7595619659175705,
 6314.4973,
 10.427811295327892,
 -24151.731167420585,
 -24151.179783105574,
 -24151.154068345782,
 -24153.03535472996,
 78.479,
 -24162.158964239457,
 -190.05999375252213,
 -191.20398763237972,
 -192.30917807576407,
 -176.45685861063248]